In [ ]:
import pandas as pd
path='s3://churn-model-data-suman/insurance-churn-2022-09-25T11-25-25/part-00000-73267f7e-66b8-4217-a095-d1e9e722aa27-c000.csv'
df = pd.read_csv(path)
df.head(2)

In [ ]:
df.isnull().sum()

In [ ]:
latest_df = df.drop(columns = ['Customer Name','Company Name'])
latest_df.shape

In [ ]:
latest_df = latest_df.fillna(df.mean)
latest_df.shape

In [ ]:
import numpy as py
x=py.array(latest_df)

In [ ]:
from sklearn.model_selection import train_test_split
x_train1, x_test = train_test_split(x,test_size = 0.05)
x_train,x_validate = train_test_split(x_train1,test_size = 0.4)
x_test_df = pd.DataFrame(x_test)
x_test_df1 = x_test_df.drop(columns = [0])
x_test = py.array(x_test_df1)

In [ ]:
x_validate.shape

In [ ]:
x_test.shape

In [ ]:
import sagemaker
import boto3
from sagemaker import Session
Sagemaker_sesion = sagemaker.Session()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
train_df = pd.DataFrame(x_train)
test_df = pd.DataFrame(x_validate)
train_df.to_csv("s3://churn-model-data-suman/xgboost-model-datasets/xgboot-train.csv",header = False, index = False)
test_df.to_csv("s3://churn-model-data-suman/xgboost-model-datasets/xgboot-test.csv",header = False, index = False)
target_path = "s3://churn-model-data-suman/xgboost-model-datasets/xgboot-train.csv"
test_path = "s3://churn-model-data-suman/xgboost-model-datasets/xgboot-test.csv"
output = "s3://churn-model-data-suman/xgboost_model-artifact/output"

In [ ]:
#pip install -U sagemaker

In [ ]:
container = sagemaker.image_uris.retrieve("xgboost",region,version = '1.0-1')

In [ ]:
XGboost_regression = sagemaker.estimator.Estimator(container,
                                                  role,
                                                  instance_count = 1,
                                                  instance_type = 'ml.m4.xlarge',
                                                   output_path = output,
                                                   sagemaker_session = Sagemaker_sesion
                                                  )
XGboost_regression.set_hyperparameters(max_depth = 10,
                                       objective = 'binary:logistic',
                                       colsample_bytree = 0.3,
                                       alpha = 2,
                                       eta = 0.3 ,
                                      num_round = 200)

In [ ]:
target_input = sagemaker.session.TrainingInput(s3_data = target_path,content_type = 'csv',s3_data_type = 'S3Prefix')
validation_input = sagemaker.session.TrainingInput(s3_data = test_path,content_type = 'csv',s3_data_type = 'S3Prefix')

In [ ]:
data_channel = {'train': target_input , 'validation' :  validation_input }
XGboost_regression.fit(data_channel)

In [ ]:
predictor = XGboost_regression.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

In [ ]:
predictor.endpoint_name

In [ ]:
x_test_df.head(20)

In [ ]:
from sagemaker.predictor import csv_serializer
predictor.serializer = csv_serializer
predictions = predictor.predict(x_test)

In [ ]:
pred_st = str(predictions).split(',')
pred_st[0] = pred_st[0][2:]
pred_st[-1] = pred_st[-1][:-1]
for i in range(len(pred_st)) :
    pred_st[i] = float(pred_st[i])

In [ ]:
import   matplotlib.pyplot as  plt
y_test_df1 = x_test_df[0]
y_test = py.array(y_test_df1)
plt.scatter(pred_st,y_test)
plt.show()

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from math import sqrt
k = x_test.shape[1]
n = len(x_test)
RMSE = float(format(py.sqrt(mean_squared_error(y_test, pred_st)),'.3f'))
MSE = mean_squared_error(y_test, pred_st)
MAE = mean_absolute_error(y_test, pred_st)
r2 = r2_score(y_test, pred_st)
adj_r2 = 1-(1-r2)*(n-1)/(n-k-1)
print('RMSE =',RMSE, '\nMSE =',MSE, '\nMAE =',MAE, '\nR2 =', r2, '\nAdjusted R2 =', adj_r2)